In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D

In [2]:
# Data
data = {
    'gejala': [
        'tidak bisa dinyalakan',
        'mesin mengeluarakan bau',
        'bunyi mesin aneh',
        'bergetar tidak normal',
        'motor panas berlebihan',
        'mengeluarkan bunyi berdecit',
        'rem tidak berfungsi',
        'motor tersendat sendat',
        'oli mengalami kebocoran',
        'overheat pada mesin',
        'warna asap putih',
        'bunyi mesin aneh',
        'motor berbunyi knocking',
        'turunnya performa mesin',
        'getaran tidak normal',
        'keluar bercak oli',
        'pengurangan level oli',
        'kebocoran blok mesin',
        'mesin tidak befungsi',
        'kondisi aki melemah',
        'lampu indikator menyala',
        'mesin sulit dihidupkan',
        'stater berbunyi aneh',
        'mesin mati mendadak',
        'mesin bergetar kuat',
        'getaran pada kemudi',
        'berisik saat berkendara',
        'warna asap hitam',
        'bau asap aneh',
        'mesin tidak berdaya',
        'berbunyi saat mengerem',
        'rem terasa licin',
        'kecepatan rem melemah',
        'terdapat gesekan gigi',
        'sulit memindahkan gigi',
        'transmisi berbunyi aneh',
        # Tambahkan gejala lain sesuai kebutuhan
    ],
    'kerusakan': [
        'Mesin',
        'Mesin',
        'Mesin',
        'Kelistrikan',
        'Kelistrikan',
        'Kelistrikan',
        'Sistem Rem',
        'Sistem Rem',
        'Sistem Rem',
        'Overheat',
        'Overheat',
        'Overheat',
        'bunyi ketukan pada mesin',
        'bunyi ketukan pada mesin',
        'bunyi ketukan pada mesin',
        'kebocoran minyak',
        'kebocoran minyak',
        'kebocoran minyak',
        'kegagalan aki',
        'kegagalan aki',
        'kegagalan aki',
        'masalah saat menyala',
        'masalah saat menyala',
        'masalah saat menyala',
        'getaran berlebihan',
        'getaran berlebihan',
        'getaran berlebihan',
        'asap dari knalpot',
        'asap dari knalpot',
        'asap dari knalpot',
        'masalah pengereman',
        'masalah pengereman',
        'masalah pengereman',
        'masalah transmisi',
        'masalah transmisi',
        'masalah transmisi',
        # Tambahkan label lain sesuai kebutuhan
    ]
}

df = pd.DataFrame(data)
df.head()


,gejala,kerusakan
0,tidak bisa dinyalakan,Mesin
1,mesin mengeluarakan bau,Mesin
2,bunyi mesin aneh,Mesin
3,bergetar tidak normal,Kelistrikan
4,motor panas berlebihan,Kelistrikan


In [23]:
# Membagi data menjadi training dan testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Tokenisasi teks
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['gejala'])
vocab_size = len(tokenizer.word_index) + 1

# Mengonversi teks menjadi sequences
X_train = tokenizer.texts_to_sequences(train_data['gejala'])
X_test = tokenizer.texts_to_sequences(test_data['gejala'])

# Padding sequences agar memiliki panjang yang sama
maxlen = max(len(seq) for seq in X_train + X_test)
X_train_padded = pad_sequences(X_train, padding='post', maxlen=maxlen) #NOTE
X_test_padded = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Label encoding untuk kolom target
label_to_index = {label: i for i, label in enumerate(df['kerusakan'].unique())}
y_train = train_data['kerusakan'].map(label_to_index)
y_test = test_data['kerusakan'].map(label_to_index)

In [24]:
# Membuat model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(label_to_index), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Melatih model
model.fit(X_train_padded, y_train, epochs=70, validation_data=(X_test_padded, y_test), batch_size=32)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 3, 100)            5300      
                                                                 
 lstm_8 (LSTM)               (None, 128)               117248    
                                                                 
 dense_8 (Dense)             (None, 12)                1548      
                                                                 
Total params: 124,096
Trainable params: 124,096
Non-trainable params: 0
_________________________________________________________________
Epoch 1/70
1/1 [==============================] - 5s 5s/step - loss: 2.4853 - accuracy: 0.1786 - val_loss: 2.4893 - val_accuracy: 0.0000e+00
Epoch 2/70
1/1 [==============================] - 0s 77ms/step - loss: 2.4795 - accuracy: 0.2143 - val_loss: 2.4924 - val_accuracy: 0.0000e+00
Epoch 3/70
1/1 [============

1/1 [==============================] - 0s 56ms/step - loss: 1.0984 - accuracy: 0.8214 - val_loss: 3.1385 - val_accuracy: 0.0000e+00
Epoch 53/70
1/1 [==============================] - 0s 68ms/step - loss: 1.0595 - accuracy: 0.8571 - val_loss: 3.1653 - val_accuracy: 0.0000e+00
Epoch 54/70
1/1 [==============================] - 0s 56ms/step - loss: 1.0201 - accuracy: 0.8571 - val_loss: 3.1902 - val_accuracy: 0.0000e+00
Epoch 55/70
1/1 [==============================] - 0s 59ms/step - loss: 0.9682 - accuracy: 0.8571 - val_loss: 3.2136 - val_accuracy: 0.0000e+00
Epoch 56/70
1/1 [==============================] - 0s 61ms/step - loss: 0.9195 - accuracy: 0.8929 - val_loss: 3.2367 - val_accuracy: 0.0000e+00
Epoch 57/70
1/1 [==============================] - 0s 58ms/step - loss: 0.8811 - accuracy: 0.8929 - val_loss: 3.2574 - val_accuracy: 0.0000e+00
Epoch 58/70
1/1 [==============================] - 0s 74ms/step - loss: 0.8379 - accuracy: 0.8929 - val_loss: 3.2764 - val_accuracy: 0.0000e+00
Epoc

In [27]:
# Cetak akurasi
print(f'Akurasi: {accuracy * 100:.2f}%')

Akurasi: 92.86%


In [28]:
import tkinter as tk
from tkinter import ttk
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assume label_to_index is defined during training
label_to_index = {label: i for i, label in enumerate(df['kerusakan'].unique())}

# Fungsi untuk mengonversi teks menjadi sequence
def text_to_sequence(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=X_train_padded.shape[1])
    return padded_sequence

# Fungsi untuk memprediksi kerusakan
def predict_damage():
    selected_gejala = [gejala[i] for i, var in enumerate(checkbox_vars) if var.get() == 1]

    # Hanya memproses jika ada gejala yang dipilih
    if selected_gejala:
        # Maksimal 3 gejala yang dicentang
        selected_gejala = selected_gejala[:3]

        # Mengonversi teks menjadi sequence
        input_text = ' '.join(selected_gejala)
        input_sequence = text_to_sequence(input_text)

        # Melakukan prediksi
        prediction = model.predict(input_sequence)

        # Mengambil indeks label dengan probabilitas tertinggi
        predicted_category_index = prediction.argmax(axis=1)[0]

        # Mendapatkan nama kategori berdasarkan indeks
        predicted_category = list(label_to_index.keys())[list(label_to_index.values()).index(predicted_category_index)]

        # Menampilkan hasil prediksi
        result_label.config(text=f"Kemungkinan Kerusakan: {predicted_category}")
    else:
        result_label.config(text="Silakan pilih setidaknya satu gejala.")

In [29]:
# Membuat GUI
root = tk.Tk()
root.title("Deteksi Kerusakan Sepeda Motor")

# List untuk menyimpan variabel checkbox
checkbox_vars = []

# Membuat checkbox untuk setiap gejala
gejala = df['gejala']
num_gejala = len(gejala)

# Menentukan jumlah kolom
num_cols = 2
# Menentukan jumlah baris (dibulatkan ke atas agar cukup untuk semua gejala)
num_rows = -(-num_gejala // num_cols)

for i, g in enumerate(gejala):
    var = tk.IntVar()
    checkbox = ttk.Checkbutton(root, text=g, variable=var)
    # Menentukan posisi baris dan kolom
    checkbox.grid(row=i // num_cols, column=i % num_cols, sticky="w", padx=10, pady=5)
    checkbox_vars.append(var)

# Tombol untuk memprediksi kerusakan
predict_button = ttk.Button(root, text="Prediksi Kerusakan", command=predict_damage)
# Menentukan posisi tombol pada baris setelah checkbox
predict_button.grid(row=num_rows, column=0, columnspan=num_cols, pady=10)

# Label untuk menampilkan hasil prediksi
result_label = ttk.Label(root, text="")
# Menentukan posisi label pada baris setelah tombol
result_label.grid(row=num_rows + 1, column=0, columnspan=num_cols, pady=5)

# Menjalankan GUI
root.mainloop()

1/1 [==============================] - 0s 39ms/step
